In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 3 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

07 November 2022 ; 10:07:40


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,Precious Achiuwa,C,23,TOR,10,0,21.8,3.0,7.8,0.385,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1
2,Dalano Banton,PG,23,TOR,9,0,7.9,1.4,2.9,0.500,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1
4,Khem Birch,C,30,TOR,4,0,8.5,0.8,1.3,0.600,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1
5,Chris Boucher,PF,30,TOR,7,0,21.3,4.7,8.4,0.559,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1
6,Justin Champagnie,SF,21,TOR,2,0,4.5,1.5,1.5,1.000,...,7.3,116.84,109.98,6.86,9.0,117.62,108.99,8.63,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Bobby Portis,25.8,9,1,MIL,19.7,2.733333,9.111111,13.3,1,2.3,3.644444,0.293169
1,Kevin Love,19.9,9,0,CLE,22.5,4.555556,9.111111,12.1,2,6.9,5.466667,0.262669
2,Collin Sexton,22.2,11,1,UTA,17.5,0.745455,5.218182,14.4,2,0.5,3.727273,0.196692
3,Delon Wright,22.8,4,0,WAS,17.1,4.100000,4.100000,6.5,9,0.1,0.000000,0.166764
4,Malcolm Brogdon,24.3,9,0,BOS,19.6,0.911111,5.466667,14.9,3,2.0,4.555556,0.147135


In [8]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)